訓練用データを使ってロジスティック回帰とランダムフォレストで判別を行い、性能の良い方でテストデータを使って判別を行うことにした。

まずロジスティック回帰で判別を行う。

1.vehicle.csvを読み込む
2.データの前処理（欠損の多い項目の削除、ダミー変数を入れる）
3.ロジスティック回帰で判別する

In [1]:
import pandas as pd
import numpy as np

In [2]:
#データを読み込む
data_train= pd.read_csv('/Users/ichinose_kaori/Downloads/M8exercise/vehicle.csv', encoding="utf-8")
data_train

,destination,passanger,weather,temperature,time,coupon,expiration,gender,age,maritalStatus,...,CoffeeHouse,CarryAway,RestaurantLessThan20,Restaurant20To50,toCoupon_GEQ5min,toCoupon_GEQ15min,toCoupon_GEQ25min,direction_same,direction_opp,Y
0,Home,Alone,Sunny,80,10PM,Carry out & Take away,1d,Female,26,Single,...,less1,4~8,1~3,never,1,0,0,1,0,1
1,No Urgent Place,Alone,Sunny,55,10AM,Coffee House,2h,Female,26,Single,...,less1,4~8,less1,4~8,1,0,0,0,1,1
2,Home,Alone,Snowy,30,6PM,Coffee House,1d,Female,46,Unmarried partner,...,less1,less1,1~3,less1,1,1,0,0,1,0
3,Home,Alone,Snowy,30,10PM,Coffee House,2h,Male,31,Married partner,...,less1,4~8,1~3,4~8,1,1,0,0,1,0
4,Home,Alone,Rainy,55,10PM,Coffee House,1d,Female,26,Single,...,less1,1~3,gt8,gt8,1,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9508,No Urgent Place,Alone,Rainy,55,10AM,Bar,1d,Female,31,Married partner,...,less1,gt8,4~8,less1,1,1,0,0,1,0
9509,No Urgent Place,Friend(s),Sunny,80,2PM,Coffee House,2h,Female,31,Unmarried partner,...,never,4~8,NaN,never,1,1,0,0,1,1
9510,Work,Alone,Sunny,55,7AM,Coffee House,1d,Female,50plus,Single,...,never,4~8,4~8,less1,1,1,0,0,1,0
9511,Work,Alone,Sunny,80,7AM,Restaurant(20-50),2h,Female,31,Single,...,less1,gt8,1~3,less1,1,0,0,1,0,0


In [3]:
# データの型と欠損の確認。
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9513 entries, 0 to 9512
Data columns (total 26 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   destination           9513 non-null   object
 1   passanger             9513 non-null   object
 2   weather               9513 non-null   object
 3   temperature           9513 non-null   int64 
 4   time                  9513 non-null   object
 5   coupon                9513 non-null   object
 6   expiration            9513 non-null   object
 7   gender                9513 non-null   object
 8   age                   9513 non-null   object
 9   maritalStatus         9513 non-null   object
 10  has_children          9513 non-null   int64 
 11  education             9513 non-null   object
 12  occupation            9513 non-null   object
 13  income                9513 non-null   object
 14  car                   86 non-null     object
 15  Bar                   9423 non-null   

In [5]:
# 列除去前の状態を確認。
data_train.dtypes

destination             object
passanger               object
weather                 object
temperature              int64
time                    object
coupon                  object
expiration              object
gender                  object
age                     object
maritalStatus           object
has_children             int64
education               object
occupation              object
income                  object
car                     object
Bar                     object
CoffeeHouse             object
CarryAway               object
RestaurantLessThan20    object
Restaurant20To50        object
toCoupon_GEQ5min         int64
toCoupon_GEQ15min        int64
toCoupon_GEQ25min        int64
direction_same           int64
direction_opp            int64
Y                        int64
dtype: object

In [6]:
# 欠損値の多いCarを削除。
data_train = data_train.drop(["car"],axis=1)
data_train.dtypes

destination             object
passanger               object
weather                 object
temperature              int64
time                    object
coupon                  object
expiration              object
gender                  object
age                     object
maritalStatus           object
has_children             int64
education               object
occupation              object
income                  object
Bar                     object
CoffeeHouse             object
CarryAway               object
RestaurantLessThan20    object
Restaurant20To50        object
toCoupon_GEQ5min         int64
toCoupon_GEQ15min        int64
toCoupon_GEQ25min        int64
direction_same           int64
direction_opp            int64
Y                        int64
dtype: object

In [7]:
#データのほとんどがObjectでダミー変数への変換が必要。楽に指定するため、データの行の一覧を取得
data_train.columns

Index(['destination', 'passanger', 'weather', 'temperature', 'time', 'coupon',
       'expiration', 'gender', 'age', 'maritalStatus', 'has_children',
       'education', 'occupation', 'income', 'Bar', 'CoffeeHouse', 'CarryAway',
       'RestaurantLessThan20', 'Restaurant20To50', 'toCoupon_GEQ5min',
       'toCoupon_GEQ15min', 'toCoupon_GEQ25min', 'direction_same',
       'direction_opp', 'Y'],
      dtype='object')

In [8]:
#ダミー変数を入れる
cat_col = ['destination', 'passanger', 'weather', 'time', 'coupon','expiration', 'gender', 'age', 'maritalStatus','education', 'occupation', 'income', 'Bar', 'CoffeeHouse', 'CarryAway','RestaurantLessThan20', 'Restaurant20To50']
data_train=pd.get_dummies(data_train,columns=cat_col)

#与えられた訓練用データをモデル作成用の訓練用データとテストデータに分ける。

In [9]:
#Yが何番目の列かを調べる
data_train.columns

Index(['temperature', 'has_children', 'toCoupon_GEQ5min', 'toCoupon_GEQ15min',
       'toCoupon_GEQ25min', 'direction_same', 'direction_opp', 'Y',
       'destination_Home', 'destination_No Urgent Place',
       ...
       'RestaurantLessThan20_1~3', 'RestaurantLessThan20_4~8',
       'RestaurantLessThan20_gt8', 'RestaurantLessThan20_less1',
       'RestaurantLessThan20_never', 'Restaurant20To50_1~3',
       'Restaurant20To50_4~8', 'Restaurant20To50_gt8',
       'Restaurant20To50_less1', 'Restaurant20To50_never'],
      dtype='object', length=110)

In [10]:
#データから訓練データとテストデータを作成。
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split( data_train, test_size = 0.2, random_state = 0)
print('訓練データ数 : {}, テストデータ数 : {}'.format(len(train_df), len(test_df)))

訓練データ数 : 7610, テストデータ数 : 1903


In [11]:
# 標準化、スケーリング、正規化を行う。

standard_f=False
if standard_f:
    from sklearn.preprocessing import StandardScaler
else:
    from sklearn.preprocessing import MinMaxScaler

# numpyの配列として値を取り出す
X_train = train_df.drop('Y', axis=1)
Y_train = train_df.iloc[:,8].values
X_test = test_df.drop('Y', axis=1)
Y_test = test_df.iloc[:,8].values

if standard_f:
    transformer = StandardScaler()
    X_train = transformer.fit_transform(X_train)# 訓練データに標準化を適用してください。10点
    X_test = transformer.transform(X_test)# テストデータに標準化を適用してください。20点
else:
    transformer = MinMaxScaler()
    X_train = transformer.fit_transform(X_train)
    X_test = transformer.transform(X_test)
                                   
normalize_f = True
if normalize_f:
    from sklearn.preprocessing import normalize
    X_train = normalize(X_train, norm='l1', axis=1)
    X_test = normalize(X_test, norm='l1', axis=1)

In [12]:
# ロジスティック回帰による学習と予測を実行します。
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
import numpy as np

params = {"C": np.logspace(0, 4, 5)}
logreg_cv = GridSearchCV(LogisticRegression(), cv=5, param_grid=params)
logreg_cv.fit(X_train, Y_train)
print('訓練データでの分類精度 : {0:.2%}'.format(logreg_cv.score(X_train, Y_train)))
print('テストデータでの分類精度 : {0:.2%}'.format( logreg_cv.score(X_test, Y_test)))

訓練データでの分類精度 : 100.00%
テストデータでの分類精度 : 100.00%


次にランダムフォレストで予測を行う。

In [13]:
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix

np.random.seed(42)

In [14]:
# データ読込
data_train = pd.read_csv('/Users/ichinose_kaori/Downloads/M8exercise/vehicle.csv', encoding="utf-8") #訓練用データ
data_train.head()

,destination,passanger,weather,temperature,time,coupon,expiration,gender,age,maritalStatus,...,CoffeeHouse,CarryAway,RestaurantLessThan20,Restaurant20To50,toCoupon_GEQ5min,toCoupon_GEQ15min,toCoupon_GEQ25min,direction_same,direction_opp,Y
0,Home,Alone,Sunny,80,10PM,Carry out & Take away,1d,Female,26,Single,...,less1,4~8,1~3,never,1,0,0,1,0,1
1,No Urgent Place,Alone,Sunny,55,10AM,Coffee House,2h,Female,26,Single,...,less1,4~8,less1,4~8,1,0,0,0,1,1
2,Home,Alone,Snowy,30,6PM,Coffee House,1d,Female,46,Unmarried partner,...,less1,less1,1~3,less1,1,1,0,0,1,0
3,Home,Alone,Snowy,30,10PM,Coffee House,2h,Male,31,Married partner,...,less1,4~8,1~3,4~8,1,1,0,0,1,0
4,Home,Alone,Rainy,55,10PM,Coffee House,1d,Female,26,Single,...,less1,1~3,gt8,gt8,1,0,0,1,0,1


In [ ]:
# 特徴データとラベルをX, yに格納
X = np.array(data_train.drop('Y', axis=1))#class(最後の１列)以外を説明変数として、numpy arrayに変換しXに代入してください。 5点
y= np.array(data_train.iloc[:, 8])#classの列をnumpy arrayに変換してyに代入してください。 1が悪性，0が良性を表します。 5点
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=15)

transformer = StandardScaler() #データを標準化するtransformer
X_train = transformer.fit_transform(X_train) #訓練データを標準化
X_test = transformer.transform(X_test) #訓練データと同じtransformerで標準化